In [ ]:
# Эта среда Python 3 поставляется со множеством полезных аналитических библиотек.
# Она определяется образом Docker kaggle/python: https://github.com/kaggle/docker-python
# Например, вот несколько полезных пакетов для загрузки.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Задача:
**предсказать жанр фильма по описанию к нему.**

В нашем распоряжении два датасета: тренировочный с 54214 фильмами и тестовый 54200.

In [ ]:
# импортируем необходимые пакеты
import pandas as pd                 # для работы с табличными данными
import matplotlib.pyplot as plt     # для визуализации
import seaborn as sns               # для визуализации

import nltk                         # библиотека для обработки естественного языка
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# загрузка в текущую директорию пакета wordnet
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

import string
import re                           # библиотека обработки регулярных выражений

from sklearn.model_selection import train_test_split         # инструмент сплитования набора данных
from sklearn.feature_extraction.text import TfidfVectorizer  # векторизация текста

from sklearn.linear_model import LogisticRegression          # алгоритм логистической регрессии
from sklearn.metrics import classification_report, ConfusionMatrixDisplay # набор метрик для задач классификации

from catboost import Pool, CatBoostClassifier               # библиотека градиентного спуска

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# загружаем тренировочный набор данных
train_data = pd.read_csv('/kaggle/input/sf-dl-movie-genre-classification/train.csv')
train_data.head()

In [ ]:
# загружаем тестовый набор данных
test_data = pd.read_csv('/kaggle/input/sf-dl-movie-genre-classification/test.csv')
test_data.head()

#### 1. Общая информация
Посмотрим, все ли фильмы содержат категорию-жанр и текстовое описание или имеются пропуски.

In [ ]:
display(train_data.info())
test_data.info()

Пропусков нет, но как распределены фильмы по жанрам?

In [ ]:
plt.figure(figsize=(20,7))
sns.countplot(train_data, x='genre')   # график для подсчета категориальных признаков
plt.xticks(rotation=60)
plt.title('Distribution of movies by genre', fontsize=16);



Итак, целевая переменная очень несбалансирована, при построении простых моделей не следует ожидать высокой точности предсказания.

#### 2. Очистка и подготовка описания фильма к созданию матрицы векторов.
Подготовим наш признак, а именно описание фильма: удалим любые числа (они не отражают категорию фильма), знаки препинания, текст в скобках, приведем текст к нижнему регистру. Также преобразуем слова с помощью леммантизации к единому формату и удалим излишние стоп-слова из текста.

In [ ]:
# функция очистки текста
def filtered_text(df, text):
    # удаляем любые числа, оставляя только английские слова, преобразуем к нижнему регистру
    df['clean_text'] = df[text].apply(lambda x: re.sub(r"[^a-z]\w*\d\w*", " ", x.lower().strip()))
    # удаляем любые дополнительные слова в скобках
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r"\(.+?\)", "", x).strip())
    # удаляем все знаки препинания
    df['clean_text'] = df['clean_text'].str.translate(str.maketrans(" ", " ", string.punctuation))
    
    # рабиваем текст на отдельные слова-токены
    df['clean_text'] = df['clean_text'].apply(lambda x:word_tokenize(x))
    # леммантизация слов
    lemmatizer = WordNetLemmatizer()
    df['clean_text'] = df['clean_text'].apply(lambda x: list(map(lambda word: lemmatizer.lemmatize(word), x)))
    # находим и удаляем стоп-слова
    stop_words = set(stopwords.words('english'))
    df['clean_text'] = df['clean_text'].apply(lambda x: list(filter
                                    (lambda word: word not in stop_words and len(word)>2, x)))
    # объединяем токены обратно в текст
    df['clean_text'] = df['clean_text'].apply(lambda x:' '.join(x))
    return df

# применим функцию к тренировочному набору 
clean_train = filtered_text(train_data, 'text')
clean_train.head()

In [ ]:
# посмотрим на один пример очищенного текста
clean_train.clean_text[5]

In [ ]:
# применим функцию очистки к тестовой выборке
clean_test = filtered_text(test_data, 'text')
clean_test.head()

Посчитаем количество уникальных слов в каждой выборке и общий размер словаря.

In [ ]:
words_train = [word for row in clean_train['clean_text'] for word in row.split(' ')]
words_test = [word for row in clean_test['clean_text'] for word in row.split(' ')]
total_voc = set(words_train+words_test)
print('Количество слов в обучающем наборе: %s' % (len(set(words_train))),
     'Количество слов в тестовом наборе: %s' % (len(set(words_test))),
     'Общее количество уникальных слов: %s' % (len(total_voc)), sep='\n')

#### 3. Создание эмбеддинга

In [ ]:
# инициализируем векторизатор 
vectorizer = TfidfVectorizer(max_features=50000) # ограничимся четвертью из общего вокабуляра
# преобразовываем оба набора данных в матрицу TF-IDF

vectorizer.fit(pd.concat([clean_train.clean_text, clean_test.clean_text], axis=0))
train = vectorizer.transform(clean_train.clean_text)
test = vectorizer.transform(clean_test.clean_text)
print('Размер выборок тренировочной и тестовой: ', train.shape, test.shape, sep='\n')

Целевую переменную закодируем порядковым номером для классификации с помощью логистической регрессии.\
Разделим тренировочный набор на обучающую и валидационную выборки.

In [ ]:
# создаем список жанров
genres = list(clean_train['genre'].unique())
# создаем словарь, где каждому жанру присваиваем порядкой номер
coding_genres = {value: num for num, value in enumerate(genres)}
display(coding_genres)
# применим созданный словарь к целевой переменной genre
y = clean_train['genre'].apply(lambda x: coding_genres[x])

# создадим обучающий и валидационный наборы с учетом стратификации для корректного распределения жанров по выборкам
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, 
                                                      random_state=42, stratify=y)                                                     

#### 4. Модель логистической регрессии

In [ ]:
# инициализируем алгоритм
clf = LogisticRegression(C=20, class_weight='balanced', multi_class='multinomial',
                         max_iter=1000, random_state=42)
# обучаем
%time clf.fit(X_train, y_train)

In [ ]:
# предсказываем
y_pred = clf.predict(X_valid)
# посмотрим значения метрик классификации на валидационной выборке
print(classification_report(y_valid, y_pred, digits=3))

In [ ]:
# визуализируем матрицу ошибок
fig, ax = plt.subplots(figsize=(15,15))
ConfusionMatrixDisplay.from_estimator(clf, X_valid, y_valid, ax=ax);
ax.set_xticklabels(list(coding_genres.keys()), rotation=90)
ax.set_yticklabels(list(coding_genres.keys()))
ax.set_title('Confusion matrix of logistic regression', fontsize=18);

Сделаем предсказание данной моделью на тестовой выборке и сохраним полученные значения в submission.

In [ ]:
y_test = clf.predict(test)

# обратная функция для преобразования целевого признака из числового в категориальный строковый
def back_genres(value):
    return [key for key in coding_genres if coding_genres[key] == value][0]

# создаем датафрейм submission с нужными колонками id и genre
submission = pd.DataFrame(y_test, columns=['genre'])
submission.index.name = 'id'
submission.index += 1

submission['genre'] = submission['genre'].apply(back_genres)
print(submission)
# сохраняем submission
submission.to_csv('submission_logreg.csv')

Простая модель средне справилась с поставленной задачей, мы попробуем также ансамблевый алгоритм - градиентный бустинг библиотеки catboost.

#### 5. Градиентный бустинг

In [ ]:
# разделим очищенный тренировочный датасет на обучающую и валидационную выборки, 
# применяя стратификацию, помним, что целевая переменная у нас сильно несбалансирована
Y = clean_train.genre
X_train, X_valid, y_train, y_valid = train_test_split(clean_train.clean_text, Y, test_size=0.2, 
                                                      random_state=42, stratify=Y) 
# проверим полученное соотношение: как целевая переменная 'genre' разделилась между выборками
df_y_train = pd.DataFrame(data=y_train.value_counts(normalize=True).round(3)).reset_index()
df_y_valid = pd.DataFrame(data=y_train.value_counts(normalize=True).round(3)).reset_index()
print(pd.concat([df_y_train, df_y_valid], axis=1))

In [ ]:
# создадим обучающий и валидационнный контейнера Pool для обучения модели, чтобы повысить производительность
train_pool = Pool(data=pd.DataFrame(clean_train['clean_text'][X_train.index]),
                  label=pd.DataFrame(clean_train['genre'][y_train.index]), 
                  text_features=['clean_text'])
valid_pool = Pool(data=pd.DataFrame(clean_train['clean_text'][X_valid.index]),
                  label=pd.DataFrame(clean_train['genre'][y_valid.index]), 
                  text_features=['clean_text'])

In [ ]:
# задаем параметры модели
catboost_params = {
    'iterations': 1000,                 # количество итераций
    'learning_rate': 0.04,              # шаг обучения
    'depth': 8,                         # макс глубина одного экземпляра-дерева
    'eval_metric': 'TotalF1',           # метрика
    'task_type': 'GPU',               
    'early_stopping_rounds': 50, 
    'use_best_model': True,
    'verbose': 100,
    'auto_class_weights': 'Balanced',
    'random_seed': 42
}
# инициализируем алгоритм и передаем ему параметры
catboost_model = CatBoostClassifier(**catboost_params)
# обучаем
catboost_model.fit(train_pool, eval_set=valid_pool)

In [ ]:
y_predicted = catboost_model.predict(valid_pool)
print(classification_report(y_valid, y_predicted, digits=3))

Делаем предсказание на тестовой выборке и также сохраняем полученные значения в submission.

In [ ]:
test_pool = Pool(data=pd.DataFrame(clean_test['clean_text']),
                      text_features=['clean_text'])
y_test_catboost = catboost_model.predict(test_pool)
submission_catboost = pd.DataFrame(y_test_catboost, columns=['genre'])
submission_catboost.index.name = 'id'
submission_catboost.index += 1
print(submission_catboost.head())
submission_catboost.to_csv('submission_catboost.csv')

Как мы видим, градиентный бустинг значительно уступил логистической регрессии в данной задаче.

#### 6. Нейронная сеть

In [ ]:
# импортируем необходимые инструменты для построения простой нейронной сети
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from keras.layers import Input, Embedding, Flatten, Dropout, MaxPooling1D
from keras.layers import Conv1D, LSTM, Dense
from keras.models import Model

In [ ]:
# закодируем one-hot-encoding целевую переменную
Y = pd.get_dummies(train_data.genre)
Y.shape

In [ ]:
max_features = 100000                     # задаем размер словаря
sequence_length = 100
embedding_size = 32

# создаем токенайзер
tokenizer = Tokenizer(
    num_words=max_features,
    # filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    # lower=True,
    split=" ",
    char_level=False,
    oov_token='OOV'
)
# используем уже подготовленный очищенный признак - описание фильма
# обучение токенайзера делаем на всем словаре из тренировочной и тестовой выборок вместе
tokenizer.fit_on_texts(clean_train.clean_text.to_list() + clean_test.clean_text.to_list())
# применим токенайзер к тренировочной и тестовой выборкам
X_train_seq = tokenizer.texts_to_sequences(clean_train.clean_text)
X_test_seq = tokenizer.texts_to_sequences(clean_test.clean_text)
# преобразуем созданные эммбединги (обучающий и тестовый) в последовательности одинакового размера
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=sequence_length)  # усечение длины предложения 100
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=sequence_length)
print('Shape train_set: ', X_train_pad.shape, 'Shape valid_set: ', X_test_pad.shape)

Разделим полученный тренировочный эммбединг на обучающий и валидационный, 90% отдадим на обучение.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_pad, Y,
                                                  stratify=Y,
                                                  random_state=42,
                                                  test_size=0.1)

In [ ]:
# построим небольшую нейронную сеть
def neural_model():
    inputs = Input(shape=[sequence_length])
    layer = Embedding(input_dim=max_features, output_dim=embedding_size, input_length=sequence_length)(inputs)
    layer = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=4)(layer)
    layer = LSTM(300, dropout=0.25, recurrent_dropout=0.25)(layer)  
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(256, activation='relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(27, activation='sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model
    

In [ ]:
model = neural_model()
model.summary()
# задаем оптимизатор, функцию потерь и метрику измерения качества
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_valid, y_valid),
                    batch_size=64)

In [ ]:
# визуализируем движение метрики в процессе обучения 
plt.style.use(['dark_background'])
fig, ax = plt.subplots(figsize=(7, 4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('MODEL ACCURACY')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Делаем предсказание на тестовой выборке и записываем результат в submission, сохраним его.

In [ ]:
%time
predictions = Y.columns[np.argmax(model.predict(X_test_pad), axis=1)]
submission_nn = pd.DataFrame({'id': range(1,test_data.shape[0]+1), 
                           'genre': predictions})
print(submission_nn.head())
submission_nn.to_csv('submission_nn.csv', index=False)


#### По итогам трех примененных алгоритмов наиболее успешной оказалась логистическая регрессия, хотя ее качество не высокое.
Для улучшения качества предсказания правильным будем поработать над самим признаком - описанием к фильму и добавить к нему определение семантического окраса.